### Creating And Visualizing Word Embeddings Using Mary Shelley's Frankenstein, Word2Vec, PyTorch and Tensorboard

In [1]:
import re
import numpy as np
import word2vec

ModuleNotFoundError: No module named 'word2vec'

In [3]:
with open('frankenstein.txt') as frank:
    txt = frank.read()

In [4]:
txt = txt.split()[1:]

In [5]:
vocab = []
for word in txt:
    if word not in vocab:
        vocab.append(word)

In [6]:
vocab = sorted(vocab)

In [7]:
parsed_vocab = []
for word in vocab: 
    parsed_word, n = re.subn('[\n.\]!_\[=\",\')?:(#$-\;]', '', word)
    parsed_vocab.append(parsed_word)

In [8]:
word2idx = {word: idx for idx, word in enumerate(parsed_vocab)}

In [9]:
word2idx['Oxford']

828

In [10]:
pairs = []
txt_len = len(txt)

for i in range(txt_len):
    word = txt[i]
    
    if i > 0:
        pairs.append((word, txt[i-1]))
    elif i > 1:
        pairs.append((word, txt[i-2]))
    elif i < txt_len-1:
        pairs.append((word, txt[i-1]))
    elif i < txt_len-2:
        pairs.append((word, txt[i-2]))

In [11]:
import torch
from torch.utils.tensorboard import SummaryWriter
from torch.nn.functional import one_hot, cross_entropy
import matplotlib.pyplot as plt
from random import shuffle

In [12]:
vocab_len = len(vocab)
embedding_dim = 50

In [13]:
device = torch.device("cpu")

In [16]:
# Embedding = torch.randn(vocab_len, embedding_dim, device=device, required_grad=True)
Prediction = torch.randn(embedding_dim, vocab_len, device=device, required_grad=True)

TypeError: randn() received an invalid combination of arguments - got (int, int, required_grad=bool, device=torch.device), but expected one of:
 * (tuple of ints size, torch.Generator generator, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)


In [168]:
def process_wordpairs(pairs):
        return torch.tensor([one_hot(word2idx[word], vocab_len) for word, context in pair] for pair in pairs).to(device), torch.tensor([word2idx[context] for word, context in pair] for pair in pairs).to(device)

In [ ]:
def do_train_iter(batch):
    word, contexts = processwordpairs(batch)
    
    vector = torch.matmul(Embedding, words)
    guesses = torch.matmul(Prediction, vectors)
    
    loss = cross_entropy(guesses, context)
    
    loss.backwards()
    
    Embedding_grad = Embedding.grad.item()
    Prediction_grad = Prediction.grad.item()
    
    Embedding -= (Embedding_grad + (Embedding_grad_momentum * momentum_rate if Embedding_grad_momentum is not None else 0)) * learning_rate
    Prediction -= (Prediction_grad + (Prediction_grad_momentum * momentum_rate if Prediction_grad_momentum is not None else 0)) * learning_rate 
    
    Embedding_grad_momentum = Embedding_grad
    Prediction_grad_momentum = Prediction_grad
    
    return loss.item()

In [171]:
tb = SummaryWriter()

In [182]:
def train(pairs, batch_size, epochs=50):
    pairs = shuffle(pairs)
    batch = []
    running_loss = 0
    num_batches = len(pairs) // batch_size
    
    for epoch in range(epochs):
        for i in range(0, len(pairs)):
            batch.append(pairs[i])
            if i+1 % batch_size == 0 or i == len(pairs):
                for idx, words in enumerate(batch):
                    loss = do_train_iter(words)
                    running_loss += loss
                    tb.add_scalar("Loss", loss, idx + epoch*num_batches)
                print("Epoch {}, Loss {}".format(epoch, running_loss/num_batches))

In [176]:
import tensorboard

In [177]:
%load_ext tensorboard

<module 'tensorboard' from '/home/herzfeld2/anaconda3/lib/python3.7/site-packages/tensorboard/__init__.py'>